# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/mamba")

Mounted at /content/gdrive


In [ ]:
%pip install triton causal-conv1d

In [4]:
%pip install --no-build-isolation --verbose .

  [3/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output '/content/gdrive/Othercomputers/retep'\''s PC/mamba/build/temp.linux-x86_64-cpython-310/csrc/selective_scan/selective_scan_bwd_fp32_real.o'.d '-I/content/gdrive/Othercomputers/retep'"'"'s PC/mamba/csrc/selective_scan' -I/usr/local/lib/python3.10/dist-packages/torch/include -I/usr/local/lib/python3.10/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/lib/python3.10/dist-packages/torch/include/TH -I/usr/local/lib/python3.10/dist-packages/torch/include/THC -I/usr/local/cuda/include -I/usr/include/python3.10 -c -c '/content/gdrive/Othercomputers/retep'\''s PC/mamba/csrc/selective_scan/selective_scan_bwd_fp32_real.cu' -o '/content/gdrive/Othercomputers/retep'\''s PC/mamba/build/temp.linux-x86_64-cpython-310/csrc/selective_scan/selective_scan_bwd_fp32_real.o' -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-

In [ ]:
import mamba_ssm

In [ ]:
!rm -rf build/

In [5]:
import torch, transformers, importlib, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Lab

In [6]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
input_ids = tokenizer("Today is Sunday. Tomorrow is", return_tensors="pt")["input_ids"].to('cuda')
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
tokenizer.batch_decode(model(input_ids).logits.argmax(-1))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/517M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

The 'batch_size' argument of MambaCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.


[', the,\n is Monday']

In [7]:
importlib.reload(InitMamba)
import InitMamba

model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
# model1.train()

res1 = model1(input_ids, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states

In [8]:
res = model1(input_ids, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
tokenizer.batch_decode(res.logits.argmax(-1))

[', the,\n is Monday']

In [10]:
ssm_last_states[0].mean().backward()

In [ ]:
# Model without initial SSM states gives random results
print(tokenizer.batch_decode(res1.logits.argmax(dim=-1)))

# Model with initial SSM states gives "Sunday"
res2 = model1(input_ids, inputs_ssm_states = ssm_last_states)
print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))

[', the,\n is Monday']
[', the,\n is Monday']


# Check same

In [ ]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(input_ids).logits - model1(input_ids).logits).abs().sum())